In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
path = "/content/drive/MyDrive/NUSFintech/src/engineered_features.csv"
df = pd.read_csv(path)
df.head()

,customerID,customerType,riskLevel,investmentCapacity,account_creation_date,ISIN,transactionID,transactionType,transaction_date,totalValue,...,days_since_last_trade,holding_duration_days,cust_asset_total_value,cust_asset_n_txn,cust_asset_avg_value,cust_asset_buy_ratio,last_cust_asset_txn,cust_asset_days_since_last_txn,cust_total_value,cust_asset_portfolio_share
0,00017496858921195E5A,Professional,Aggressive,CAP_GT300K,2021-03-19,GRS434003000,7590224,Buy,2020-03-27,11000.0,...,0,1793,229180.0,12,19098.333333,0.5,2021-05-19,560,728451.013,0.314613
1,00017496858921195E5A,Professional,Aggressive,CAP_GT300K,2021-03-19,GRS434003000,7607029,Sell,2020-04-06,12080.0,...,0,1793,229180.0,12,19098.333333,0.5,2021-05-19,560,728451.013,0.314613
2,00017496858921195E5A,Professional,Aggressive,CAP_GT300K,2021-03-19,GRS434003000,7634872,Buy,2020-04-24,13400.0,...,0,1793,229180.0,12,19098.333333,0.5,2021-05-19,560,728451.013,0.314613
3,00017496858921195E5A,Professional,Aggressive,CAP_GT300K,2021-03-19,GRS434003000,7652627,Sell,2020-05-07,12700.0,...,0,1793,229180.0,12,19098.333333,0.5,2021-05-19,560,728451.013,0.314613
4,00017496858921195E5A,Professional,Aggressive,CAP_GT300K,2021-03-19,GRS434003000,7664807,Buy,2020-05-15,12150.0,...,0,1793,229180.0,12,19098.333333,0.5,2021-05-19,560,728451.013,0.314613


In [4]:
df.columns

Index(['customerID', 'customerType', 'riskLevel', 'investmentCapacity',
       'account_creation_date', 'ISIN', 'transactionID', 'transactionType',
       'transaction_date', 'totalValue', 'units', 'channel', 'marketID',
       'assetName', 'assetCategory', 'assetSubCategory', 'sector', 'industry',
       'asset_description', 'exchangeID', 'name', 'market_description',
       'country', 'tradingHours', 'marketClass', 'days_since_txn',
       'value_per_unit', 'is_buy', 'is_sell', 'channel_encoded',
       'account_age_days', 'risk_encoded', 'capacity_numeric',
       'n_unique_assets', 'n_unique_sectors', 'n_unique_markets',
       'total_invested', 'avg_txn_value', 'median_units', 'n_transactions_x',
       'n_buys_x', 'n_sells_x', 'buy_ratio_x', 'channel_pref', 'last_txn_date',
       'days_since_last_txn', 'risk_level', 'invest_capacity',
       'portfolio_concentration', 'day_of_week', 'month', 'quarter',
       'n_unique_investors', 'total_traded_value', 'avg_traded_value',
      

In [5]:
df_pos = df[df["is_buy"]==1].copy()
df_pos["user_id"] = df_pos["customerID"]
df_pos["item_id"] = df_pos["ISIN"]
ts = pd.to_datetime(df_pos["transaction_date"], errors="coerce")
df_pos = df_pos[ts.notna()].copy()
df_pos["timestamp"] = ts[ts.notna()].astype("int64") // 10**9
df_pos["rating"] = 1
df_pos = df_pos[["user_id", "item_id", "timestamp", "rating"]].sort_values("timestamp")
df_pos = df_pos.drop_duplicates(["user_id", "item_id"], keep="last")


In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

In [7]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
DEVICE

device(type='cuda')

In [9]:
EMBED_GMF = 32
EMBED_MLP = 32
MLP_LAYERS = (128, 64, 32)
NEG_PER_POS = 4
BATCH_SIZE = 2048
EPOCHS = 10
LR = 1e-3
WEIGHT_DECAY = 1e-6
TOPK = 10

In [10]:
df_cf = df_pos[df_pos["rating"]>0].copy()
test_df = df_cf.groupby("user_id").tail(1)
train_df = df_cf.drop(test_df.index)

In [11]:
user_all = pd.concat([train_df["user_id"], test_df["user_id"]])
item_all = pd.concat([train_df["item_id"], test_df["item_id"]])
user2id = {u: i for i, u in enumerate(user_all.astype("category").cat.categories)}
item2id = {i: j for j, i in enumerate(item_all.astype("category").cat.categories)}
train_df["u"] = train_df["user_id"].map(user2id).astype(int)
train_df["i"] = train_df["item_id"].map(item2id).astype(int)
test_df["u"] = test_df["user_id"].map(user2id).astype(int)
test_df["i"] = test_df["item_id"].map(item2id).astype(int)

/tmp/ipython-input-3811300264.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["u"] = test_df["user_id"].map(user2id).astype(int)
/tmp/ipython-input-3811300264.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["i"] = test_df["item_id"].map(item2id).astype(int)


In [12]:
n_users = len(user2id)
n_items = len(item2id)
print(n_users, n_items)

29090 320


In [13]:
user_pos = defaultdict(set)
for u,i in zip(train_df.u.values, train_df.i.values):
    user_pos[int(u)].add(int(i))

In [14]:
class PairwiseImplicitDataset(Dataset):
    def __init__(self, df, n_items, user_pos, neg_per_pos=4):
        import numpy as np
        self.user_features = ["risk_encoded", "capacity_numeric", "channel_encoded", "account_age_days"]
        self.item_features = ["assetCategory", "sector", "industry", "marketClass"]

        missing_user = [c for c in self.user_features if c not in df.columns]
        missing_item = [c for c in self.item_features if c not in df.columns]
        for c in ["u", "i"]:
            if c not in df.columns:
                missing_user.append(c)
        if missing_user or missing_item:
            raise ValueError(f"Missing columns. user: {missing_user}, item: {missing_item}")

        df = df.reset_index(drop=True)

        self.users = df["u"].to_numpy(dtype=np.int64)
        self.items = df["i"].to_numpy(dtype=np.int64)
        self.uf_mat = df[self.user_features].to_numpy(dtype=np.float32)
        self.if_mat = df[self.item_features].to_numpy(dtype=np.float32)

        self.n_items = int(n_items)
        self.user_pos = user_pos
        self.neg_per_pos = int(neg_per_pos)

    def __len__(self):
        return self.users.shape[0]

    def __getitem__(self, idx):
        import numpy as np
        u = int(self.users[idx])
        p = int(self.items[idx])
        uf = self.uf_mat[idx]
        ifeat = self.if_mat[idx]

        # negative sampling
        liked = self.user_pos.get(u, set())
        negs = []
        if len(liked) < self.n_items - 1:
            comp = np.setdiff1d(
                np.arange(self.n_items, dtype=np.int64),
                np.fromiter(liked, dtype=np.int64, count=len(liked)),
                assume_unique=False
            )
            comp = comp[comp != p]
            if comp.size >= self.neg_per_pos:
                negs = np.random.choice(comp, size=self.neg_per_pos, replace=False).astype(np.int64)
            else:
                negs = np.random.choice(comp, size=self.neg_per_pos, replace=True).astype(np.int64)
        else:
            while len(negs) < self.neg_per_pos:
                j = np.random.randint(0, self.n_items, dtype=np.int64)
                if j != p and j not in liked:
                    negs.append(j)
            negs = np.asarray(negs, dtype=np.int64)

        return (
            torch.as_tensor(u, dtype=torch.long),
            torch.as_tensor(p, dtype=torch.long),
            torch.as_tensor(negs, dtype=torch.long),
            torch.from_numpy(uf),
            torch.from_numpy(ifeat),
        )


In [15]:
print(train_df.columns.tolist())


['user_id', 'item_id', 'timestamp', 'rating', 'u', 'i']


In [16]:
# Feature Prep + Safe Merges (drop-in replacement)
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np

# 1) Declare feature columns
user_features = ["risk_encoded", "capacity_numeric", "channel_encoded", "account_age_days"]
item_features = ["assetCategory", "sector", "industry", "marketClass"]

# 2) Work on a copy of the master table to avoid side effects
df_enc = df.copy()

# 3) Encode item categorical columns if needed (no-op for numeric)
for col in item_features:
    if df_enc[col].dtype == "object":
        le = LabelEncoder()
        df_enc[col] = le.fit_transform(df_enc[col].astype(str))

# 4) Build per-user table and scale user features WITHOUT leakage
user_base = df_enc.drop_duplicates("customerID")[["customerID"] + user_features].copy()
scaler = MinMaxScaler()

# Fit on users present in train_df, then transform all users
train_users = train_df["user_id"].unique()
mask_train = user_base["customerID"].isin(train_users)

user_base.loc[mask_train, user_features] = scaler.fit_transform(
    user_base.loc[mask_train, user_features]
)
if (~mask_train).any():
    user_base.loc[~mask_train, user_features] = scaler.transform(
        user_base.loc[~mask_train, user_features]
    )

# 5) Build per-item table (already numerically encoded above)
item_base = df_enc.drop_duplicates("ISIN")[["ISIN"] + item_features].copy()

# 6) Clean old columns (prevents _x/_y suffix conflicts on reruns)
train_df = train_df.drop(columns=user_features + item_features, errors="ignore").copy()
test_df  = test_df.drop(columns=user_features + item_features, errors="ignore").copy()

# 7) Merge user + item features into train/test
train_df = train_df.merge(user_base, left_on="user_id", right_on="customerID", how="left")
train_df = train_df.merge(item_base, left_on="item_id", right_on="ISIN", how="left")
train_df.drop(columns=["customerID", "ISIN"], inplace=True, errors="ignore")
train_df[user_features + item_features] = (
    train_df[user_features + item_features].fillna(0).astype("float32")
)

test_df = test_df.merge(user_base, left_on="user_id", right_on="customerID", how="left")
test_df = test_df.merge(item_base, left_on="item_id", right_on="ISIN", how="left")
test_df.drop(columns=["customerID", "ISIN"], inplace=True, errors="ignore")
test_df[user_features + item_features] = (
    test_df[user_features + item_features].fillna(0).astype("float32")
)

# 8) Sanity checks
assert set(user_features) <= set(train_df.columns), "User features missing in train_df"
assert set(item_features) <= set(train_df.columns), "Item features missing in train_df"
assert set(user_features) <= set(test_df.columns),  "User features missing in test_df"
assert set(item_features) <= set(test_df.columns),  "Item features missing in test_df"

print("Feature merge complete.")


Feature merge complete.


/tmp/ipython-input-209357280.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.5 1.  0.5 ... 0.  0.  0. ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  user_base.loc[mask_train, user_features] = scaler.fit_transform(
/tmp/ipython-input-209357280.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.16908898 0.2112726  0.2112726  ... 0.2112726  0.2112726  0.28571429]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  user_base.loc[mask_train, user_features] = scaler.fit_transform(


In [17]:
# make sure these exist: train_df, n_items, user_pos, NEG_PER_POS, BATCH_SIZE
train_df = train_df.reset_index(drop=True)

train_loader = DataLoader(
    PairwiseImplicitDataset(train_df, n_items, user_pos, NEG_PER_POS),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=False
)


In [18]:
class NeuMF(nn.Module):
    def __init__(self, n_users, n_items, eg=32, em=32, layers=(128,64,32)):
        super().__init__()
        # GMF branch
        self.user_gmf = nn.Embedding(n_users, eg)
        self.item_gmf = nn.Embedding(n_items, eg)
        # MLP branch
        self.user_mlp = nn.Embedding(n_users, em)
        self.item_mlp = nn.Embedding(n_items, em)
        dim = em*2
        mlp = []
        for h in layers:
            mlp += [nn.Linear(dim, h), nn.ReLU()]
            dim = h
        self.mlp = nn.Sequential(*mlp)
        # fusion
        self.out = nn.Linear(eg + layers[-1], 1)
        self.sig = nn.Sigmoid()
        self._init()
    def _init(self):
        for emb in [self.user_gmf, self.item_gmf, self.user_mlp, self.item_mlp]:
            nn.init.normal_(emb.weight, std=0.01)
        for m in self.mlp:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)
        nn.init.xavier_uniform_(self.out.weight); nn.init.zeros_(self.out.bias)
    def forward(self, u, i, uf=None, ifeat=None):
      g = self.user_gmf(u) * self.item_gmf(i)
      m = self.mlp(torch.cat([self.user_mlp(u), self.item_mlp(i)], dim=-1))

      if uf is not None and ifeat is not None:
          x = torch.cat([g, m, uf, ifeat], dim=-1)
          out = self.out_ext(x)
      else:
          out = self.out(torch.cat([g, m], dim=-1))

      return self.sig(out.squeeze(-1))


In [19]:
def bce_pos_neg(model, users, pos_items, neg_items):
    B, K = neg_items.shape
    users = users.to(DEVICE); pos_items = pos_items.to(DEVICE); neg_items = neg_items.to(DEVICE)
    pos_scores = model(users, pos_items)
    users_rep = users.unsqueeze(1).expand(-1, K).reshape(-1)
    neg_scores = model(users_rep, neg_items.reshape(-1)).view(B, K)
    loss = nn.functional.binary_cross_entropy(pos_scores, torch.ones_like(pos_scores))
    loss += nn.functional.binary_cross_entropy(neg_scores, torch.zeros_like(neg_scores))
    return loss

In [20]:
@torch.no_grad()
def evaluate(model, test_df, user_pos, n_items, k=10, n_neg=99):
    model.eval()
    HR, NDCG = [], []
    for u, pos in zip(test_df.u.values, test_df.i.values):
        u = int(u); pos = int(pos)
        negs = []
        s = user_pos[u]
        while len(negs) < n_neg:
            j = np.random.randint(0, n_items)
            if j != pos and j not in s:
                negs.append(j)
        items = np.array([pos] + negs, dtype=np.int64)
        users = np.full_like(items, u)
        scores = model(
            torch.tensor(users, device=DEVICE),
            torch.tensor(items, device=DEVICE)
        ).cpu().numpy()
        rank = scores.argsort()[::-1]
        topk = items[rank[:k]]
        HR.append(1 if pos in topk else 0)
        if pos in topk:
            idx = np.where(topk == pos)[0][0]
            NDCG.append(1 / np.log2(idx + 2))
        else:
            NDCG.append(0.0)
    return float(np.mean(HR)), float(np.mean(NDCG))


In [23]:

model = NeuMF(n_users, n_items, EMBED_GMF, EMBED_MLP, MLP_LAYERS).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

for e in range(1,  16):
    model.train()
    total = 0.0

    for u, p, n, _, _ in train_loader:
        u = u.long().to(DEVICE)
        p = p.long().to(DEVICE)
        n = n.long().to(DEVICE)

        opt.zero_grad()
        loss = bce_pos_neg(model, u, p, n)
        loss.backward()
        opt.step()
        total += loss.item() * len(u)

    # Evaluate after each epoch (features not used by model)
    hr, ndcg = evaluate(model, test_df, user_pos, n_items, k=TOPK)

    print(f"Epoch {e:02d} | train_bce={total/len(train_df):.4f} | "
          f"HR@{TOPK}={hr:.3f} | NDCG@{TOPK}={ndcg:.3f}")


Epoch 01 | train_bce=1.2546 | HR@10=0.609 | NDCG@10=0.327
Epoch 02 | train_bce=0.8193 | HR@10=0.663 | NDCG@10=0.414
Epoch 03 | train_bce=0.7367 | HR@10=0.640 | NDCG@10=0.407
Epoch 04 | train_bce=0.6992 | HR@10=0.654 | NDCG@10=0.411
Epoch 05 | train_bce=0.6718 | HR@10=0.674 | NDCG@10=0.419
Epoch 06 | train_bce=0.6514 | HR@10=0.677 | NDCG@10=0.420
Epoch 07 | train_bce=0.6248 | HR@10=0.679 | NDCG@10=0.422
Epoch 08 | train_bce=0.5963 | HR@10=0.668 | NDCG@10=0.419
Epoch 09 | train_bce=0.5654 | HR@10=0.667 | NDCG@10=0.421
Epoch 10 | train_bce=0.5306 | HR@10=0.686 | NDCG@10=0.434
Epoch 11 | train_bce=0.5004 | HR@10=0.715 | NDCG@10=0.450
Epoch 12 | train_bce=0.4755 | HR@10=0.722 | NDCG@10=0.463
Epoch 13 | train_bce=0.4517 | HR@10=0.740 | NDCG@10=0.475
Epoch 14 | train_bce=0.4322 | HR@10=0.742 | NDCG@10=0.474
Epoch 15 | train_bce=0.4114 | HR@10=0.733 | NDCG@10=0.477


In [24]:
import torch

checkpoint = {
    "model_state_dict": model.state_dict(),
    "user2id": user2id,
    "item2id": item2id,
    "params": {
        "n_users": n_users,
        "n_items": n_items,
        "eg": EMBED_GMF,
        "em": EMBED_MLP,
        "layers": MLP_LAYERS,
    },
}

torch.save(
    checkpoint,
    "/content/drive/MyDrive/NUSFintech/models/neumf_checkpoint.pth"
)
print("Saved NeuMF checkpoint")

Saved NeuMF checkpoint


In [25]:
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ckpt = torch.load(
    "/content/drive/MyDrive/NUSFintech/models/neumf_checkpoint.pth",
    map_location=DEVICE,
)

params = ckpt["params"]
n_users = params["n_users"]
n_items = params["n_items"]

model = NeuMF(
    n_users=n_users,
    n_items=n_items,
    eg=params["eg"],
    em=params["em"],
    layers=tuple(params["layers"]),
).to(DEVICE)

model.load_state_dict(ckpt["model_state_dict"])
model.eval()

user2id = ckpt["user2id"]
item2id = ckpt["item2id"]

print("Loaded NeuMF checkpoint")


Loaded NeuMF checkpoint


In [37]:
import torch

some_user_id = list(user2id.keys())[0]    # e.g. '00017496858921195E5A'
some_item_id = list(item2id.keys())[0]    # first item in your mapping

print("Chosen user_id:", some_user_id)
print("Chosen item_id:", some_item_id)

u_idx = torch.tensor([user2id[some_user_id]], dtype=torch.long, device=DEVICE)
i_idx = torch.tensor([item2id[some_item_id]], dtype=torch.long, device=DEVICE)

model.eval()
with torch.no_grad():
    score = model(u_idx, i_idx)

print("User:", some_user_id)
print("Item:", some_item_id)
print("Predicted score:", score.item())


Chosen user_id: 00017496858921195E5A
Chosen item_id: 100974271034
User: 00017496858921195E5A
Item: 100974271034
Predicted score: 0.8712116479873657


In [29]:
list(user2id.items())[:10]

[('00017496858921195E5A', 0),
 ('00024864C985E72167A0', 1),
 ('0004718496C71D464F57', 2),
 ('000676D07A4CF7526ECB', 3),
 ('000900E880281981624D', 4),
 ('00090203A8C62E135D0B', 5),
 ('001270E70A4ECA1CD664', 6),
 ('001387DFC77BD44899B2', 7),
 ('0017D195DFD87A91C59D', 8),
 ('001A82FE1591AD3926C7', 9)]